In [13]:
import pandas as pd
import numpy as np
from datamodel import TradingState, Listing, OrderDepth

In [14]:
data = pd.read_csv('results/zero-trades.csv', delimiter=';')

In [15]:
data.head(10)

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-1,0,PEARLS,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0
1,-1,0,BANANAS,4946,1,4945.0,30.0,NaN,NaN,4952,30,NaN,NaN,NaN,NaN,4949.0,0.0
2,-1,100,PEARLS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,-1,100,BANANAS,4945,31,NaN,NaN,NaN,NaN,4950,7,4952.0,31.0,NaN,NaN,4947.5,0.0
4,-1,200,BANANAS,4945,22,NaN,NaN,NaN,NaN,4951,1,4952.0,21.0,NaN,NaN,4948.0,0.0
5,-1,200,PEARLS,9996,1,9995.0,21.0,NaN,NaN,10004,1,10005.0,21.0,NaN,NaN,10000.0,0.0
6,-1,300,PEARLS,9996,2,9995.0,23.0,NaN,NaN,9998,3,10004.0,2.0,10005.0,23.0,9997.0,0.0
7,-1,300,BANANAS,4945,25,NaN,NaN,NaN,NaN,4952,25,NaN,NaN,NaN,NaN,4948.5,0.0
8,-1,400,PEARLS,9998,5,9996.0,2.0,9995.0,23.0,10004,2,10005.0,23.0,NaN,NaN,10001.0,0.0
9,-1,400,BANANAS,4946,5,4945.0,25.0,NaN,NaN,4952,25,NaN,NaN,NaN,NaN,4949.0,0.0


In [16]:
datad = data.to_dict("records")

In [20]:
trading_states_d = {}
for d in datad:
    timestamp = d['timestamp']
    product = d['product']

    buy_orders = {}
    sell_orders = {}
    # bids
    for i in range(3):
        price = 'bid_price_'+str(i)
        volume = 'bid_volume_'+str(i)
        buy_orders[price] = volume
    # asks
    for i in range(3):
        price = 'ask_price_'+str(i)
        volume = 'ask_volume_'+str(i)
        sell_orders[price] = volume
    
    listing = Listing(
        product,
        product,
        product
    )

    order_depth = OrderDepth(
        buy_orders,
        sell_orders
    )

    if timestamp not in trading_states_d:
        trading_states_d[timestamp] = TradingState(
            timestamp,
            {},
            {},
            {},
            {},
            {},
            {}
        )

    trading_states_d[timestamp].listings[product] = listing
    trading_states_d[timestamp].order_depths[product] = order_depth

print(list(trading_states_d))